# Running the Quantum Volume Algorithm
This example walks through the steps of running the quantum volume algorithm on square matrices. It is intended to mirror Algorithm 1 of https://arxiv.org/pdf/1811.12926.pdf. In general, we will generate a model circuit, classically compute its Heavy Output Group, then run various samplers (currently, ideal and noisy simulators) to evaluate how often they generate Heavy results.

In [ ]:
import cirq
from examples.advanced import quantum_volume
import numpy as np

# Some set-up variables. Feel free to play with these.
num_repetitions = 11 # This is supposed to be >= 100.
depths = range(2, 5) # The depths and number of qubits

# Here is the important set-up: The samplers. These are what will be
# run on the generated circuit, and then evaluated.
class SamplerConfig:
    def __init__(
        self,
        *,  # Forces keyword args.
        sampler: cirq.Sampler,
        label: str,
        color: str,
        marker: str = None):
            self.sampler = sampler
            self.label = label
            self.color = color
            self.marker = marker      
            # Keep track of the probabilities.
            self.probabilities = [0] * len(depths)
samplers = [SamplerConfig(
    label = 'Ideal simulation',
    sampler = cirq.Simulator(),
    marker = '+',
    color = 'tab:green',
), SamplerConfig(
    label = 'Noisy simulation',
    sampler = cirq.DensityMatrixSimulator(noise=cirq.ConstantQubitNoiseModel(qubit_noise_gate=cirq.DepolarizingChannel(p=0.005))),
    color = 'tab:red',
)]

for depth in depths:
    num_qubits = depth # Square matrix.
    print(f"Running simulation with {num_qubits} qubits and a depth of {depth}")
    for i in range(num_repetitions):
        print(f"    Repetition {i + 1} of {num_repetitions}")
        # Generate a model circuit.
        model_circuit = quantum_volume.generate_model_circuit(
            num_qubits, depth, random_state=np.random.RandomState()
        )
        # Compute its heavy set.
        heavy_set = quantum_volume.compute_heavy_set(model_circuit)
        print(f"        Heavy Set: {heavy_set}")
        
        # Run the given samplers over the model circuit.
        for config in samplers:
            probability = quantum_volume.sample_heavy_set(model_circuit, heavy_set, sampler=config.sampler)
            config.probabilities[depth - len(depths)] += probability
            print(f"        {config.label} HOG probability: {probability}")
        
        # TODO: The next steps are to compile the model circuit and run it on various devices.

    # Compute the average performance over the total number of runs.
    for config in samplers:
        config.probabilities[depth - len(depths)] /= num_repetitions
        print(f"    Average {config.label} HOG probability: {config.probabilities[depth - len(depths)]}")


In [ ]:
from matplotlib import pyplot as plt
from matplotlib.ticker import FormatStrFormatter

# Create a chart that is designed to look as similar as possible to
# Figure 2 in https://arxiv.org/pdf/1811.12926.pdf.
fig, axs = plt.subplots()
for idx, config in enumerate(samplers):
    axs.scatter([d + idx / 10 for d in depths], config.probabilities, marker=config.marker, c=f'{config.color}', label=f'{config.label}')
# Line markers for asymptotic ideal heavy output probability and the ideal Heavy Output Generation threshold.
axs.axhline((1 + np.log(2)) / 2, color='tab:green', label='Asymptotic ideal', linestyle='dashed')
axs.axhline(2/3, label='HOG threshold',  color='k', linestyle='dotted')
# Making the plot look consistent.
axs.set_ybound(0.4, 1)
axs.xaxis.set_major_locator(plt.MultipleLocator(1))
axs.set_xlabel("width/depth of model circuit m=d")
axs.set_ylabel("est. heavy output probability h(d)")
fig.suptitle('Experimental data for square quantum volume circuits')
axs.legend(loc='lower left')
